In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta                                   

In [2]:
NGR=pd.read_csv('playerstats.csv')

In [3]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [4]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [5]:
NGR=NGR.loc[(NGR["Advertiser"]!="crmbonus") & (NGR["Advertiser"]!="pine10")]

In [6]:
NGR["Advertiser"].unique()

array(['ally0004', 'ally0034', 'ally0032', 'ally0031', 'ally0011',
       'ally0005', 'rubju300', 'jompa', 'ally0012', 'jafe1000', 'thago',
       'rspn2', 'ally0023', 'ally0036', 'rspcom', 'franber', 'ally0022',
       'ally0015', 'ally0009', 'rspn10', 'micinf003', 'ally0037',
       'ally0003', 'ally0001', 'ally0007', 'expx8030', 'Imperador23',
       'ally0006', 'rspn9', 'ally0029', 'deifes', 'Gn8383', 'ally0018',
       'jeh50', 'ally0040', 'rspn5', 'rspn3', 'ally0002', 'pafeit',
       'rspn1', 'sixpoint02', 'biar16', 'rspn8', 'rspn7', 'ally00010'],
      dtype=object)

In [7]:
NGR = NGR.loc[(NGR["First deposit"].dt.year == 2024)]

In [8]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [9]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [10]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [11]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [12]:
output

,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,83.25,640.00,13
1,02/2024,424.25,5457.52,97
2,03/2024,1890.01,16568.08,103
3,04/2024,4691.92,5772.00,97
4,05/2024,2769.04,20044.46,288
5,06/2024,8026.40,29770.97,454
6,07/2024,23235.11,43669.72,524
7,08/2024,28578.67,70266.66,972
8,09/2024,-7055.13,64815.40,1051
9,10/2024,29647.92,98880.02,1728


In [13]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]
output


,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,83.25,640.00,13
1,02/2024,424.25,5457.52,97
2,03/2024,1890.01,16568.08,103
3,04/2024,4691.92,5772.00,97
4,05/2024,2769.04,20044.46,288
5,06/2024,8026.40,29770.97,454
6,07/2024,23235.11,43669.72,524
7,08/2024,28578.67,70266.66,972
8,09/2024,-7055.13,64815.40,1051
9,10/2024,29647.92,98880.02,1728


In [ ]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Aplicando a conversão e formatando
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

df['Data'] = "19/11/2024"
df['Ordem'] = range(1, len(df) + 1)


df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD,Data,Ordem
0,01/2024,"R$83,25","R$640,00",13,"$14,40","$110,72",18/11/2024,1
1,02/2024,"R$424,25","R$5.457,52",97,"$73,40","$944,15",18/11/2024,2
2,03/2024,"R$1.890,01","R$16.568,08",103,"$326,97","$2.866,28",18/11/2024,3
3,04/2024,"R$4.691,92","R$5.772,00",97,"$811,70","$998,56",18/11/2024,4
4,05/2024,"R$2.769,04","R$20.044,46",288,"$479,04","$3.467,69",18/11/2024,5
5,06/2024,"R$8.026,40","R$29.770,97",454,"$1.388,57","$5.150,38",18/11/2024,6
6,07/2024,"R$23.235,11","R$43.669,72",524,"$4.019,67","$7.554,86",18/11/2024,7
7,08/2024,"R$28.578,67","R$70.266,66",972,"$4.944,11","$12.156,13",18/11/2024,8
8,09/2024,"R$-7.055,13","R$64.815,40",1051,"$-1.220,54","$11.213,06",18/11/2024,9
9,10/2024,"R$29.647,92","R$98.880,02",1728,"$5.129,09","$17.106,24",18/11/2024,10


In [15]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats.xlsx",index=False)